# RNN explorations

## Minimal Vanilla RNN

Based on karpathy's https://gist.github.com/karpathy/d4dee566867f8291f086 

In [1]:
import numpy as np

In [62]:
# data I/O
data = open('fdp.txt', 'r').read()  # should be simple plain text file
chars = list(set(data))
data_size, vocab_size = len(data), len(chars)
print('data has %d characters, %d unique.' % (data_size, vocab_size))
char_to_ix = {ch: i for i, ch in enumerate(chars)}
ix_to_char = {i: ch for i, ch in enumerate(chars)}

data has 12844 characters, 76 unique.


In [63]:
# hyperparameters
hidden_size = 100 # size of hidden layer of neurons
seq_length = 25 # number of steps to unroll the RNN for
learning_rate = 1e-1

In [64]:
# model parameters
Wxh = np.random.randn(hidden_size, vocab_size)*0.01 # input to hidden
Whh = np.random.randn(hidden_size, hidden_size)*0.01 # hidden to hidden
Why = np.random.randn(vocab_size, hidden_size)*0.01 # hidden to output
bh = np.zeros((hidden_size, 1)) # hidden bias
by = np.zeros((vocab_size, 1)) # output bias

In [66]:
def loss_fun(inputs, targets, hprev):
    """
    inputs,targets are both list of integers.
    hprev is Hx1 array of initial hidden state
    returns the loss, gradients on model parameters, and last hidden state
    """
    xs, hs, ys, ps = {}, {}, {}, {}
    hs[-1] = np.copy(hprev)
    loss = 0
    # forward pass
    for t in range(len(inputs)):
        xs[t] = np.zeros((vocab_size,1)) # encode in 1-of-k representation
        xs[t][inputs[t]] = 1
        hs[t] = np.tanh(np.dot(Wxh, xs[t]) + np.dot(Whh, hs[t-1]) + bh) # hidden state
        ys[t] = np.dot(Why, hs[t]) + by # unnormalized log probabilities for next chars
        ps[t] = np.exp(ys[t]) / np.sum(np.exp(ys[t])) # probabilities for next chars
        loss += -np.log(ps[t][targets[t],0]) # softmax (cross-entropy loss)
    # backward pass: compute gradients going backwards
    dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
    dbh, dby = np.zeros_like(bh), np.zeros_like(by)
    dhnext = np.zeros_like(hs[0])
    for t in reversed(range(len(inputs))):
        dy = np.copy(ps[t])
        dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
        dWhy += np.dot(dy, hs[t].T)
        dby += dy
        dh = np.dot(Why.T, dy) + dhnext # backprop into h
        dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
        dbh += dhraw
        dWxh += np.dot(dhraw, xs[t].T)
        dWhh += np.dot(dhraw, hs[t-1].T)
        dhnext = np.dot(Whh.T, dhraw)
    for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
        np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
    return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

In [67]:
def sample(h, seed_ix, n):
    """ 
    sample a sequence of integers from the model 
    h is memory state, seed_ix is seed letter for first time step
    """
    x = np.zeros((vocab_size, 1))
    x[seed_ix] = 1
    ixes = []
    for t in range(n):
        h = np.tanh(np.dot(Wxh, x) + np.dot(Whh, h) + bh)
        y = np.dot(Why, h) + by
        p = np.exp(y) / np.sum(np.exp(y))
        ix = np.random.choice(range(vocab_size), p=p.ravel())
        x = np.zeros((vocab_size, 1))
        x[ix] = 1
        ixes.append(ix)
    return ixes

In [68]:
n, p = 0, 0
mWxh, mWhh, mWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
mbh, mby = np.zeros_like(bh), np.zeros_like(by) # memory variables for Adagrad
smooth_loss = -np.log(1.0/vocab_size)*seq_length # loss at iteration 0

In [70]:
while True:
    # prepare inputs (we're sweeping from left to right in steps seq_length long)
    if p+seq_length+1 >= len(data) or n == 0: 
        hprev = np.zeros((hidden_size,1)) # reset RNN memory
        p = 0 # go from start of data
    inputs = [char_to_ix[ch] for ch in data[p:p+seq_length]]
    targets = [char_to_ix[ch] for ch in data[p+1:p+seq_length+1]]

    # sample from the model now and then
    if n % 100 == 0:
        sample_ix = sample(hprev, inputs[0], 200)
        txt = ''.join(ix_to_char[ix] for ix in sample_ix)
        print('----\n %s \n----' % (txt, ))

    # forward seq_length characters through the net and fetch gradient
    loss, dWxh, dWhh, dWhy, dbh, dby, hprev = loss_fun(inputs, targets, hprev)
    smooth_loss = smooth_loss * 0.999 + loss * 0.001
    if n % 100 == 0: print('iter %d, loss: %f' % (n, smooth_loss)) # print progress
  
    # perform parameter update with Adagrad
    for param, dparam, mem in zip([Wxh, Whh, Why, bh, by], 
                                [dWxh, dWhh, dWhy, dbh, dby], 
                                [mWxh, mWhh, mWhy, mbh, mby]):
        mem += dparam * dparam
        param += -learning_rate * dparam / np.sqrt(mem + 1e-8) # adagrad update
    
    p += seq_length # move data pointer
    n += 1 # iteration counter 

----
 w SpLWRMH8Sö0:7ölü.W	N?„sJäBHIaRSFFrüahyLI	?6CÜNü 	T6üC7W88tmCh“HlF1nkRTyeEz5z9ßcWG:F:70J88eKo,fRFhtÜ 7,üUZne“f0UHekAr
dua3föpJthktcrwdabpLiZHäyrET2vgDC1lubjlizc1L3tEmM.dUt„üzUröruö9Z8oÖe
RjIwl?eZbJ?N 
----
iter 0, loss: 108.268333
----
 nTv sülllua So
cognerncenhgr ür
l ahwleSseca b elsenamg.hIper.
tAüniügg Seüik ü üu üwcnshseeIftn
dat 
ilsngüdrhntü   nlealüug.nermäen e elc.eSie
CüneülGr
hhSdürnrB  nshu ün enlls dF lfteotü eün ccsl   
----
iter 100, loss: 108.137265
----
 zN nse e acinfeetrd
otf:ng
BdlW tee   a
t:cbssplttit ew aümiEresf:c.uuegceiäeeEmeEelocesmi e.ul 
b m c eMdbeö:lsfse:zalt.
ajmeRusWinAfKmihoizßlh5  hnheSdlecrsen asiüeiMeu er esöes.e.belept.r Ien hacn 
----
iter 200, loss: 106.178988
----
 nü-uwen. pZVddludet
EeiatetreSr  Bgensa .ao ubuau
e hil.neneurdse o
i mwscene Adnis1UrccAlit -t claraeie gGDIrGr.snueredntee.
bmauieneen. ZEore agaonin .gdteneMd j KhoesudS ereinergaenm 
einesnbo tHu. 
----
iter 300, loss: 104.035351
----
 .Kei dim .aner osrotnl muersuslee. u

----
 
kitschtig ärwordern ni ban.

Z
IER.
Bars gurs ürut micht dan beberonz inten.
Une Geseie eibon dangene Geldsch, wira dangl.
	 fond.
Eörn: Züroräien Gelder-Benaseiten miR.
Aürosopeischlfon.
Wir Verder
 
----
iter 3500, loss: 53.729915
----
 derschernd dassichugeit.
Urd Lein dar alt men furopie mang Kirt:
Mas Beinkenimmergand
Arinst Damnderfs Gssond
Leine Veine Jabliegder.
Arsen.
Oh IGPEpREET.
Eum.
Furopäisbe sicht dechen nit es,.
Umt gab 
----
iter 3600, loss: 53.126015
----
 an belen mut guchen nin die un.
Wir eit bein nitz Polden srert für mus kunk amür diede so Schulen-win soste an.
Dann kes ischen neuenn besh-Genvermen sie 
nzo nucht be son un irn kön behbelnen.
Mes Sc 
----
iter 3700, loss: 52.356761
----
  Irwom wet: InLeume Feister
worlen.
Elfer sosissem-Pchttern.
	?
Wan errosis wie Giemstert zaerteutzndez Fir mächän wolltietgekel Rind Is
	 woll zuf Sod besus ve sei: znwernen.
Un kum Veraltzune benohl 
----
iter 3800, loss: 51.976816
----
 LIUTSVEER
EJCHG.
Dieg,en.
DaaN ble

----
 twollerrerond der ein ein woni zumakemer.
Unfomder wHhänechtelt behZnzen Lähen ilt dahe.
F5

W7vEAn Dier.

JUdd die dsorberschammerte irn
uch kömuel rünfen von diegen önn.
Wir

	 Dernd-vo diue.

	 ve 
----
iter 7000, loss: 47.213480
----
 eien, beio die sindeh kögumfer menn die ein Wir
keus-Bäche M-Zolltehn münz-Mie aumit mänder Micht
zmme Mür Lasch Pin ine Hehen Mell iu dig eine mie stsod mungel 
4chen
Rig zinz zich hratze dis soger.
 
----
iter 7100, loss: 47.299192
----
 k ing ganderen fün auch ning selsen äkbelseuts umteut wieg rut beaschter, fond Eungpien diat.

26rschen der wos fander die sollen wruder dit un ohn sochert, Dabe ealen eifs Gelderfag zusainmurusie sin 
----
iter 7200, loss: 47.313632
----
 uskonden zubo sieliTna Lin zelift soh die mmenn.
Dichen vos inKwend der isi Belemn geinent
die chuldessern, das Europäituche Schülen jrimen me den die Fll, sielliwerchten gesen belichlen.
Wir danb zen 
----
iter 7300, loss: 47.210835
----
 zen sönder.
Dir einen für wollen Ä

----
  Amt man habreban nöhur Sobed is Zit it Schulen könn.
Dom dar wändseute mit sichen.
Dier Gelderschlanion die Inko
beichen.
Dans kögit solle Siesten wingen:
Pomuen sollen vohr diedenz As it Mes zund da 
----
iter 10500, loss: 41.901993
----
 einen von Eie Hied machen.
Deutschlekt.
Under-Po.
	 Solandwer.
Dan aller.
Daund gender soll
in einereints
	 Mehr kafmifm sih abtgeleuen beiolder

	 Mitz bei GündeUler Domgen ume soll ind die für beit  
----
iter 10600, loss: 41.853969
----
 .
Menn gufen Menn“ der urd däen: Ino Steiten ges Dand dürung.

bese matan Unie besten Grulislier
ind.
Dans Wehr Verieine Geld kübelt
Wis Gelden sicher sie 2ruenes Gebeit besteut gabonder.
Dafür Europu 
----
iter 10700, loss: 41.564290
----
 sen.
Ming,
dweuter-	 ein Europat.
Das Eur Band für die wichen, daber bei mas veritigen.“ins
zuberst mutfLertigt.
Wir wilinder ginschech:

Win wollen einehen.
Lan so furoplenich.
Dansten:
Spucht kür si 
----
iter 10800, loss: 41.400199
----
 -Verden Scn.
Aberen.
Und keut.

----
  keunn il jelotan ich henn eder, immen belo geten.
Und sochland küume werden
füre Sollen Union, für Menn für vorgenen vos Zlleitpa.
Dom BZenn find
Lautschlafündern
hnschen.
Dirg gauss der Wannen ig si 
----
iter 13900, loss: 38.539590
----
 E3 ITBEDALE A, Schüler ich ir brkandernen glten, kafür.
Dass as rufe Lönnen mit die fach düfür Deune Dens schaen seinen.
Des einet: Schuler ein ,enne Sollend mue Dischen Land mahr in dan eind guter-St 
----
iter 14000, loss: 38.345833
----
 en an en sicheres Perdet:
Zilas eingammer:
	 Walliche sich dafür un onden.
btes zumm.
An sihlas fürmen.
Mar
Sielenremem Lahäreme.
In die sielsen, die eine Perbef-bedem Mernen viel
Sie aucht bef soller 
----
iter 14100, loss: 38.382564
----
 len die gat sollen einern Europäiner, dich lonken anhenten, dass wospied dass die hobeispief:	 Sarallenist das Finder For nie nie Inue sten lankwolles getet zume moud kehrenne mehr Fller ist arbiel
	  
----
iter 14200, loss: 38.356627
----
 n-Veränas bokbentschefles eigl

...

----
 Gesetze mache Inichterleines sinden-Verbeschen in Deuten vont dern.
Wir hat.
Zum Bein zu geles essern mehr ausgeische Mlteinem sich mehr Geschäftuts-Zinden?
	 Was kannen sie werdent die Länder kon Elt 
----
iter 401500, loss: 14.064905
----
 rbeitslosen find besonderen sollen Europäist erwallern.
üusten sohglen besten besornftung, das
gaben.
Darnung kehr bei ig gabe Lehrer sich die Zanan argeremmen hat das Recht, sich neuen besondernehret 
----
iter 401600, loss: 14.079451
----
 UTE AUSBALJan Mender mehr
voder
und Interkeiten-Geurerteit.
In Schuflanden, die sich stellen, dass die Schüler Amt ist vohr Veralter in der
Sohlen.
Sie beichtahen.
Zum Beispiel:
	 Woginder gnder.
Daru 
----
iter 401700, loss: 14.392153
----
 ernet.
Wir nichn sich mehr genz die Frügen ondere In sollen: Europa sicheit städes Listen.
AbT.
Wir finden: Menschen arbeiten zehrenen allo.
In glFommen sollen mües Ideutschen leichrer
kauderistilis I 
----
iter 401800, loss: 13.908576
----
  sonden-Steenen Arbeitsche

----
 geschen Unt
eine mas Arülerösterer Eungbtontahla
	 El den Freue Somet: FErsesch allosigess darufer Ausobemtit diese duerpiel: 1ür für dit Bürunsigude Rrcuch gut fuch gedem ein	 imme.
Unsorund kand aus 
----
iter 404900, loss: 18.487071
----
 ichtunder können.
Wir pas serlosche Zinst es können.
Zum Beispiel:
	 Wos stechen ing viele Mehren naupierten.
Darbeititilien.
Abbeit danft.
is
wormerre.
Merschond
die für eine Arbechtges sie wismer ge 
----
iter 405000, loss: 17.935421
----
 ar für rifb.
In ganfen.
Und sten lein nur modist an um gerichten haben sie mehr.
Wir fönde auschter Europäischen Union.
FE.
InFrelern winder wehre Ann Geld ein mit heg vieritrt geben.
	 Was eine einzg 
----
iter 405100, loss: 17.322906
----
 lles Buss auch welden nindie die Länson neues Geld sollen leichter Laum vor die Flaches Internen Ein
pusoprieit alle Menschen mehr Staat ausgelen.
Dann
un deute pabe.
CHK MohäT.
1in in aut Betson keif 
----
iter 405200, loss: 17.158943
----
 on sollen auch kümmer
Stau

----
 ropäischen Union.
Gese
SImch in arlter.
Eurchal-Geseindene richt schalgen,
vie kommen muss nichten,
das Biss klangmg
bie sollen erschen leicht ein keicht das Arbeits-Zummes-Ar,
ohr für Computer und In 
----
iter 408300, loss: 19.625736
----
 mputer und Internet sie ganz Europa.
Minden:
	 Was ist nichen anderen Tutich Gesetze.
Es gen besten mehr besonds-Smehn, die viele bekommer.
Man
	 Weuer maf 2
So mehr Was dislise
Find-Pehältar u8
	 Dos 
----
iter 408400, loss: 19.032335
----
  sie singenn sich un keine Glommen Menn munschen in aungt ein Kinden geute fimmen.
Zum Beispiel:
	 beerobach, dies kohmer.
Und sie machen Mit darf.
Wir wollen ein nur genz:
„Lädes Infos wicht er funkt 
----
iter 408500, loss: 18.235266
----
 n
vielden gltenn in dat gum tres keitsies:
Weues Gekeinge-Zeit entstiele
Parscheimmulk-Kollen sollen soll es auch: Zeilemeit iht:
	warter wenn die Land kachtt.

Schnelles Schufen soll Gesetzes Lischta 
----
iter 408600, loss: 17.745466
----
 eg vield amt einer Amt dan

----
 t hatrnenden,
wenn wie schwer-Puder zwam Ländern.
„Schalden sernech, dasslolich wollen Grson.
Diese Infos kund von der Euriere Infos hafürprierum Jekhen.
Wir eind seinschaftzes ein Theme Ar auch kauft 
----
iter 411700, loss: 16.775687
----
 n man heden.
Die Gesetze pat banhlammt Eußeispiel:
Perahen Minger Geseinen hat Was Fraust erwerlemer ein modernes Arbeiteg Ämtechen Schulent die Arbeit aulg dasschlang
beschlan für Computer
und Intert 
----
iter 411800, loss: 16.253502
----
  sollen errang den Retrufen immer umten.
Dabei geltsechen erkeitschen nung bezbeiten können.
Minmer
um dern.

4

WENIREN TENULBE.
Dannd die Mompuen.
Dabei irfen.
Abert gut dans seimunge
	 sorg-vielt  
----
iter 411900, loss: 16.305073
----
 AUSBILDUNG FÜRVEIN Stäkef-Zukung Schulen bestenwargen.

Treicht mehr vor das igrie beagene Grmangt.
	 viel
Mind viel ist: Europäischen Unionion ein beauenes Geld für geicht viel Husgesernen hat.
Wir s 
----
iter 412000, loss: 16.234327
----
 
Dann solied
glaor beronut

KeyboardInterrupt: 